# Model to predict color from RGB value

Feedforward neural network for supervised learning of rgb color values

## Imports

In [ ]:
from model.classes import Neuron, Layer, Network_Model
from model import helpers as hlp
import rgb.rgb_helpers as chlp
import pandas as pd

## Preprocess input data

The `pandas`-Library is used to preprocess input data.
Firstly, data is read from an Excel-file.

In [ ]:
training_orig_df = pd.read_excel('datasets\\rgb.xlsx')
training_orig_df

In [ ]:
training_df = training_orig_df.copy()
training_df.drop(columns=['Name'], inplace=True)

The data is then split into training and validation data using the `split_training_val_data()` function. Here a percentage can be set.

In [ ]:
train_df, val_df = hlp.split_training_val_data(training_df, 0.2)

The training and validation data are then split into input and output data using the `split_input_output_data()` function. The column headers of the training input have to specified.

In [ ]:
x_columns = ['R', 'G', 'B']
y_columns = ['Beige', 'Yellow', 'Orange', 'Red', 'Purple', 'Blue', 'Green', 'Grey', 'Brown', 'White', 'Black']

train_df_x, train_df_y = hlp.split_input_output_data(train_df, x_columns, y_columns)
val_df_x, val_df_y = hlp.split_input_output_data(val_df, x_columns, y_columns)

In [ ]:
train_df_x_norm = hlp.normalize(train_df_x, 0, 255, -1, 1)
val_df_x_norm = hlp.normalize(val_df_x, 0, 255, -1, 1)

In [ ]:
train_df_x_norm

## Instanciate a model

`add_layer()` adds a new layer to the model. The amount of neurons and the desired activation function can be set. Layer 0 is automatically set to be the input layer. Weights and biases are randomly assigned.

In [ ]:
mdl_rgb = Network_Model()
mdl_rgb.add_layer(3)
mdl_rgb.add_layer(16, activation_function='sigmoid')
mdl_rgb.add_layer(11, activation_function='sigmoid')

Using `plot_network()`, the Model can be visualized.

In [ ]:
mdl_rgb.plot_network()

Using `get_weights()`, the weights can be displayed. Weihts of row 0 are weights of the bias neuron of the respective layer.

In [ ]:
mdl_rgb.layers[1].get_weights()

Using the `predict()` method, a prediction is made based on a given input-vector. Since the model is not trained yet, the output is random.

In [ ]:
rgb = [255, 0, 0]
input = chlp.rgb_to_norm(rgb)
pred = mdl_rgb.predict(input)
print(y_columns[pred.index(max(pred))])
chlp.plot_color(input)

## Training the model

Using *backpropagation of error* the model can be trained using the `train()` method. Arguments:
- **train_df_x**: Training set *p* of type *pandas.Dataframe*<br>
- **train_df_y**: Training input *t* of type *pandas.Dataframe*<br>
- **mode**: Mode of error evaluation, can be 'online' or 'offline', of type *string*<br>
- **epochs**: Number of epochs during the training, of type *int*<br>
- **learning_rate**: Learning rate $ \eta $ of the training session, $ 0 < \eta < 1 $, of type *float*

In [ ]:
history = mdl_rgb.train(train_df_x_norm, train_df_y, epochs=200, debug=True, shuffle=True, learning_rate_p=1.3, learning_rate_n=0.7, momentum_factor=0.6)

#### Learning curve

The cumulative error over time can be plotted using the `plot_learning_curve()` method from the `helpers` module.

In [ ]:
hlp.plot_learning_curve(history, 'Epochs', 'Average Error', 'Learning Curve')

#### Prediction

Using the trained model, more accurate predictions can now be made.

In [ ]:
rgb = [140, 50, 180] # https://www.w3schools.com/colors/colors_rgb.asp
input = chlp.rgb_to_norm(rgb)
pred = mdl_rgb.predict(input) # Possible Color Predictions: Beige, Yellow, Orange, Red, Purple, Blue, Green, Grey, Brown, White, Black
print(y_columns[pred.index(max(pred))])
chlp.plot_color(input)